In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
def get_frames_as_matrix(video_path):
    cap = cv.VideoCapture(video_path)
    frames = []
    # counter = 5
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # if not start:
        #     start -= 1
        #     continue
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frames.append(frame)
        # counter -= 1

    cap.release()
    return np.array(frames)

In [3]:
frames = get_frames_as_matrix("cctvfootage.mp4")


In [4]:
# plt.imshow(frames[200], cmap='gray')
# plt.axis('off')
# plt.show()

In [5]:
def vectorize_matrix(frames, n_jobs=-1):
    # result = frames[0].flatten(order='F')
    # for i in tqdm(range(1, len(frames))):
    #     flattened = frames[i].flatten(order='F')
    #     result = np.vstack((result, flattened))
    flattened_frames = Parallel(n_jobs=n_jobs)(
        delayed(lambda frame: frame.flatten(order='F'))(frame) for frame in frames
    )
    return np.vstack(flattened_frames).T

Y = vectorize_matrix(frames[400: 405])
Y.shape

(2073600, 5)

In [6]:
def solve_optimization(Y, lam):
    m, n = Y.shape
    X = cp.Variable((m, n))
    Z = cp.Variable((m, n))

    objective = cp.normNuc(X) + lam * cp.norm(Z, 1)
    constraints = [Y == X + Z]

    problem = cp.Problem(cp.Minimize(objective), constraints)
    objective_value = problem.solve()

    return X.value, Z.value, objective_value

In [7]:
x, z, opt = solve_optimization(Y, 1 / (np.sqrt(Y.shape[0])))

: 